In [4]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:red">ch2.Ollama_LLM활용의기본개념(LangChain)</span>
# 1. LLM을 활용하여 답변 생성하기
## 1) Ollama를 이용한 로컬 LLM 이용
 - 성능은 GPT, CLaude 같은 모델보다 떨어지나, 개념설명을 위해 open source 모델 사용
 - ollama.com 다운로드 -> 설치 -> 모델 pull
     - cmd창이나 powershell 창에 ollama pull deepseek-r1:1.5b https://docs.langchain.com/oss/python/integrations/chat/ollama

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:1.5b")
result = llm.invoke("What is the captital of Korea?")
result

## 모델 pull
 - cmd창이나 powershell창(window키+R에서 powershell)에서 ollama pull llama3.2:1b

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("What is the capital of Korea?")
result

In [ ]:
result.content

In [ ]:
result = llm.invoke("일본 수도가 어디예요?")
result.content

## 2) openai 활용
 - pip install langchain-openai
 - https://auth.openai.com/log-in

In [9]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
#os.getenv("OPENAI_API_KEY")

True

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano")
result = llm.invoke("What is the capitaal of Korea? Return the name of the city only.")

# 2. 렝체인 스타일로 프롬프트 작성
 - 프롬프트 : llm호출시 쓰는 질문

In [12]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
# llm.invoke(0)
# PromptValue, str, BaseMessages리스트

## 1) 기본 프롬프트 템플릿 사용
 - PromptTemplate을 사용하여 변수가 포함된 템플릿을 작성하면 PromptValue

In [13]:
from langchain_core.prompts import PromptTemplate
llm = ChatOllama(model="llama3.2:1b")
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables = ["country"]
)
# country = input('어느 나라의 수도를 알고 싶으신가요?')
prompt = prompt_template.invoke({"country":"Korea"})
print(prompt)
llm.invoke(prompt)

text='What is the capital of Korea?'


AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that North Korea also claims Pyongyang as its capital.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:07:58.0160114Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2649207100, 'load_duration': 1606706900, 'prompt_eval_count': 32, 'prompt_eval_duration': 191722900, 'eval_count': 25, 'eval_duration': 700020600, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01b9-3be5-7161-9dee-106c804d9995-0', usage_metadata={'input_tokens': 32, 'output_tokens': 25, 'total_tokens': 57})

## 2) 메세지 기반 프롬프트 작성
 - BaseMessage 리스트
 - BaseMessage 상속받은 클래스 : AIMessage, HumanMessage, SystemMessage, ToolMessage
 - vscode에서 ctrl+shift+p : python:select interpreter입력 -> python환경선택
 - vscode에서 커널 선택

In [14]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of Korea?")
]
llm.invoke(message_list)

AIMessage(content="I think there may be a small mistake here. The capital of South Korea is Seoul, and the capital of North Korea is Pyongyang.\n\nIt's possible you're thinking of Japan, whose capital is Tokyo. Would you like more information about either Japan or Korea?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:08:45.142407Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2440738200, 'load_duration': 96016700, 'prompt_eval_count': 86, 'prompt_eval_duration': 433491200, 'eval_count': 53, 'eval_duration': 1657664300, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01b9-f4cc-78c1-8aac-603adbc6361d-0', usage_metadata={'input_tokens': 86, 'output_tokens': 53, 'total_tokens': 139})

## 3) ChatPromptTemplate 사용
 - BaseMessage리스트 -> 튜플리스트

In [15]:
# 위의 BaseMessage 리스트를 수정
# PromptTemplate : 프롬프트에 변수포함, 
# ChatPromptTemplate : SystemPrompt설정(페르소나), few shot설정, 변수포함
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
chatPrompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpfull assistant!"),
    ('human', "What is the capital of Italy?"), # 모범질문
    ("ai", "The capital of Italy is Rome."),    # 모범답안
    ('human', "What is the capital of France?"), # 모범질문
    ("ai", "The capital of France is Paris."),    # 모범답안
    ("human", "What is the capital of {country}?")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPrompt_template.invoke({"country": country})
#print("프롬프트 : ", prompt, type(prompt))
result = llm.invoke(prompt)
result.content

어느 나라 수도가 궁금하세요일본


'The capital of Japan is Tokyo.'

In [17]:
chatPromptTemplate = ChatPromptTemplate.from_messages([
    ("system", "당신은 대한민국 정보 전문 도우미입니다"),
    ("human", "{country}의 수도가 어디예요!")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPromptTemplate.invoke({"country":country})
# print(prompt)
result = llm.invoke(prompt)
print(result.content)

어느 나라 수도가 궁금하세요한국
한국의 수도는 Seoul입니다. 그것은 한국 전역에서 가장 큰 도시이며, 그 때에 이웃된 다른 도시를 상당히 outnumber했습니다.


In [18]:
result

AIMessage(content='한국의 수도는 Seoul입니다. 그것은 한국 전역에서 가장 큰 도시이며, 그 때에 이웃된 다른 도시를 상당히 outnumber했습니다.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:41:27.0456109Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1492017800, 'load_duration': 134154300, 'prompt_eval_count': 43, 'prompt_eval_duration': 199166300, 'eval_count': 34, 'eval_duration': 953342600, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01d7-e82f-7e00-9c2d-aef0e4b0d6bb-0', usage_metadata={'input_tokens': 43, 'output_tokens': 34, 'total_tokens': 77})

# 3. 답변 형식 컨트롤 하기
 - llm.invoke()의 결과는 AIMessage() -> string이나 json, 객체 : OutputParser이용
 ## 1) 문자열 출력 파서 이용
  - StrOutputParser를 사용하여 LLM출력(AIMessage)을 단순 문자열로 변환

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
# 명시적인 지시하상이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
# 프롬프트 템플릿에 값 주입
prompt = prompt_template.invoke({"country": "Korea"})
# print(prompt)
ai_message = llm.invoke(prompt)
# print(ai_message)
# 문자열 출력 파서를 이요하여 llm응답(AIMessage객체)을 단순 문자열로 변환
output_parser = StrOutputParser()
result = output_parser.invoke(ai_message)
result

'Seoul'

In [24]:
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

In [34]:
# 변수설정, system, few shot 지정
chat_prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant with expertise in South Korea."),
    ('human', "What is the capital of Italy?"), # 모범질문
    ("ai", "Rome."),    # 모범답안
    ('human', "What is the capital of France?"), # 모범질문
    ("ai", "Paris."),    # 모범답안
    ("human", "What is the capital of {country}?")#Return the name of the city only
])
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(chat_prompt_template.invoke({"country":"Korea"})))

'The capital of South Korea is Seoul, and the administrative division that includes Pyongyang in North Korea is known as Kaesong Pukguksa ( Province of Kaesong)'

## 2) Json 출력 파서 이용
 - {'name':'홍길동', 'age':22}

In [35]:
from langchain_core.output_parsers import JsonOutputParser
country_detail_prompt = PromptTemplate(
    template = """Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
prompt = country_detail_prompt.invoke({"country":"Korea"})
ai_message = llm.invoke(prompt)
# print(ai_message.content)
output_parser = JsonOutputParser()
json_result = output_parser.invoke(ai_message)
print(json_result, type(json_result))

{'capital': 'Seoul', 'population': 51.8, 'language': 'Korean', 'currency': 'Korean won'} <class 'dict'>


In [36]:
output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country":"Japen"})))

{'capital': 'Tokyo',
 'population': '127,112,533',
 'language': 'Japanese',
 'currency': 'Japan yen'}

## 3) 구조화된 출력 사용
 - Pydantic 모델을 사용하여 LLM출력을 구조화된 형식으로 받기(JsonParser보다 훨씬 안정적)
 - Pydantic : 데이터유효성검사, 설정관리를 간편하게 해주는 라이브러리

In [37]:
class User:
    def __init__(self, id, name, is_active=True):
        self.id   = id
        self.name = name
        self.is_active = is_active
user = User("1", "홍길동", False)
print(user)

In [38]:
from pydantic import BaseModel, Field
class User(BaseModel):
    # gt=0:id>0 / ge=0:id>=0 / lt=0:id<0 / le=0:id<=0
    id:int   = Field(gt=0,         description="id")
    name:str = Field(min_length=2, description="name")
    is_active:bool = Field(default=True, description="id활성화 여부")
user = User(id="1", name="홍길동")
print(user)

id=1 name='홍길동' is_active=True


In [39]:
country_detail_prompt = PromptTemplate(
    template = """Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
class CountryDetail(BaseModel): #description: 더 정확한 출력 유도
    capital:str  = Field(description="the capital of the country")
    population:int = Field(description="the population of the country")
    language:str = Field(description="the language of the country")
    currency:str = Field(description="the currency of the country")
# 출력 형식 파서 + LLM
structedllm = llm.with_structured_output(CountryDetail)
# llm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info = structedllm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info

CountryDetail(capital='Seoul', population=51, language='Korean', currency='Korean won')

In [40]:
type(info)

__main__.CountryDetail

In [41]:
info.capital, info.population, info.language, info.currency

('Seoul', 51, 'Korean', 'Korean won')

In [42]:
print("info를 json 스타일로 :", info.model_dump_json())
print("info를 dict로 :", info.model_dump())
print("info를 dict로 :", info.__dict__)

info를 json 스타일로 : {"capital":"Seoul","population":51,"language":"Korean","currency":"Korean won"}
info를 dict로 : {'capital': 'Seoul', 'population': 51, 'language': 'Korean', 'currency': 'Korean won'}
info를 dict로 : {'capital': 'Seoul', 'population': 51, 'language': 'Korean', 'currency': 'Korean won'}


In [43]:
type(info.model_dump_json())

str

# 4. LCEL을 활용한 LangChain 생성하기
## 1) 문자열 출력 파서 이용
 - invoke
 - StrOutputParser, ChatOllama, PromptTemplate등은 모두 Runnable로부터 상속 받음.

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",
                temperature=0) # 일관된 답변(보수적인 답변)
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
output_parser = StrOutputParser() # AIMessage()를 Str변환
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) LCEL을 사용한 간단한 체인 구성
 - 파이프연산자(|) 이용

In [46]:
# 프롬프트 템플릿 -> llm -> 출력파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser
# 생성된 체인 invoke
capital_chain.invoke({"country":"Korea"})

'Seoul'

## 3) 복합체인 구성
 - 여러 단계의 추론이 필요한 경우(체인 연결)

In [48]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""Guess the name of the country based on the following informat:
    {information}
    Return the name of the country only""",
    input_variables=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "This country is very famous for its wine"})))

'Italy'

In [49]:
# 나라명 추출 체인 생성
country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information":"This country is very famous for its wine"})

'Italy'

In [50]:
# 복합체인 : 나라설명 -> 나라명(country_chain)
#                        나라명 -> 수도(capital_chain)
final_chain = country_chain | capital_chain
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

In [ ]:
# 복합체인 : information -> country_chain -> (나라명을 country) -> capital_chain
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | \
                {"country":country_chain} | capital_chain

In [ ]:
final_chain.invoke("This country is very famous for its wine")

 - 한글 지원이 안 되는 모델은 렝체인 연결이 잘 안 됨

In [51]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""다음의 {information} 설명을 보고 나라이름을 맞춰봐:
    {information}
    나라 이름만 한국어로 reutrn 해 줘""",
    input_variables=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "이 나라는 와인으로 유명해"})))

# 5. 생성형 AI 평가: 나라명을 입력하면 그 나라의 제일 유명한 음식의 레시프를 출력하는 복합체인 구현
 - 위 : 나라설명->나라이름->수도 (체인두개 연결)
 - 나라이름 -> 그 나라에서 제일 유명한 음식 -> 레시피
 - exaone3.5:2.4b, llama3.1:405b(수능 언어영역 2위), llama3:70b(llama3.1:405b와 유사) llama3.2:3b

In [ ]:
## 복합체인 : 나라명 -> 그 나라에서 제일 유명한 음식(food_chain)
##                     음식 -> 레시피(recipe_chain)

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [3]:
food_chain = ChatOllama(model="exaone3.5:2.4b",
                temperature=0
                )

prompt_template = PromptTemplate(
    template="{국가}에서 가장 유명한 음식 한가지만 답해. 음식명만 답해.",
    input_variables=["국가"]
)

recipe_prompt = ChatOllama(model="exaone3.5:2.4b",
                temperature=0,
                template="{음식}: 레시피를 알려줘",
                input_variables=["음식"]
                )

output_parser = StrOutputParser()
recipe_chain = prompt_template | food_chain | output_parser | recipe_prompt | output_parser

print(recipe_chain.invoke({"국가":"한국"}))

비빔밥은 한국의 대표적인 전통 음식 중 하나로, 다양한 재료들이 조화롭게 어우러져 풍부한 맛과 영양을 제공하는 요리입니다. 주요 구성 요소와 특징은 다음과 같습니다:

### 주요 재료:
1. **밥**: 보통 백미를 사용하지만, 건강을 위해 현미나 잡곡밥을 선택하기도 합니다.
2. **주요 채소**:
   - **시금치**: 신선하고 영양가가 높습니다.
   - **당근**: 단맛과 함께 비타민 A가 풍부합니다.
   - **파프리카**: 다양한 색상으로 시각적 매력과 비타민 C가 풍부합니다.
   - **무**: 상큼한 맛과 식감을 더합니다.
   - **오이**: 시원한 맛과 수분 공급에 좋습니다.
3. **고기**:
   - **소고기**: 주로 사용되며, 간장으로 양념합니다.
   - **닭고기**: 저지방 고단백 식품으로 선택되기도 합니다.
   - **두부**: 채식주의자를 위한 대안으로 인기가 있습니다.
4. **기타 재료**:
   - **고추장**: 매콤한 맛의 핵심 재료입니다.
   - **간장**: 깊은 감칠맛을 더합니다.
   - **참기름**: 고소한 풍미를 더합니다.
   - **깨소금**: 약간의 소금과 함께 사용하여 맛을 균형 있게 맞춥니다.
   - **김가루**: 고소한 풍미와 식감을 더합니다.

### 조리 방법:
1. **밥 준비**: 밥을 그릇에 담습니다.
2. **고기 조리**: 선택한 고기를 간장과 고추장으로 양념하여 볶거나 익힙니다.
3. **채소 준비**: 각 채소를 적당한 크기로 자르고 볶거나 살짝 데쳐서 준비합니다.
4. **조합**: 밥 위에 준비한 고기와 채소들을 고루 얹습니다.
5. **마무리**: 고추장 양념을 곁들여 비벼 먹습니다. 참기름과 깨소금을 뿌려 마무리합니다.

비빔밥은 개인의 취향에 따라 재료를 조절할 수 있어 매우 유연하고 창의적인 요리입니다. 다양한 계절 재료를 활용하면 더욱 풍성한 맛을 즐길 수 있습니다.
